# Rolling up Offer and Transaction Data for Clustering 

In [1]:
import pandas as pd
import os
import pandasql as ps

In [2]:
os.listdir()

['3. Clustering and LDA.ipynb',
 '2. Modeling.ipynb',
 '1. Initial Analysis and Preprocessing.ipynb',
 'data']

## Query for Offer Related Information

In [3]:
offers = pd.read_csv('data/offers_transformed.csv')

In [4]:
offers.head()

,Unnamed: 0,person,received,received time,offer_id,viewed,viewed time,completed,completed time,reward,transaction,transaction time,amount
0,0,78afa995795e4d85b5d9ceeca43f5fef,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,6.0,1,132.0,5.0,1,132.0,19.89
1,1,a03223e636434f42ac4c3df47e8bac43,1,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,6.0,0,NaN,NaN,0,NaN,NaN
2,2,a03223e636434f42ac4c3df47e8bac43,1,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,624.0,0,NaN,NaN,0,NaN,NaN
3,3,e2127556f4f64592b11af22de27a7932,1,0,2906b810c7d4411798c6938adc9daaa5,1,18.0,0,NaN,NaN,0,NaN,NaN
4,4,8ec6ce2a7e7949b1bf142def7d0e0586,1,0,fafdcd668e3743c1bb461111dcafc2a4,1,12.0,0,NaN,NaN,0,NaN,NaN


In [5]:
query = """
SELECT person, IFNULL(SUM(viewed), 0) as total_offer_views, IFNULL(SUM(completed), 0) as total_completed_offers, 
IFNULL(SUM(reward), 0) as total_rewards_received, IFNULL(AVG(amount), 0) as average_offer_payment
FROM offers
GROUP BY person
"""

offer_rollup = ps.sqldf(query = query)

## Transaction Queries

In [6]:
transactions = pd.read_csv("transactions_transformed.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'transactions_transformed.csv'

In [ ]:
num_transactions = transactions.groupby(by = 'person').sum()[['transaction']].reset_index().rename({'transaction':'number_of_transactions'}, axis = 1)

In [ ]:
query = """
SELECT person, IFNULL(AVG(amount), 0) as average_transaction_amount
FROM transactions
GROUP BY person
"""
average_transactions = ps.sqldf(query = query)

## Join Data Together

In [ ]:
query = """
SELECT t1.person as person_id, * 
FROM offer_rollup t1 JOIN num_transactions t2 on t1.person = t2.person 
JOIN average_transactions t3 ON t1.person = t3.person
"""

full_profile_clustering = ps.sqldf(query = query)

In [ ]:
full_profile_clustering.drop(columns = 'person', inplace = True)

## Checking Correlation

In [ ]:
# check for correlations briefly
full_profile_clustering.corr(numeric_only=True)

## Export Data

In [ ]:
full_profile_clustering.to_csv('user_activity_rollup.csv', index = False)

## KMeans Clustering

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#bring in rolled up data by person id
cluster_data = pd.read_csv("user_activity_rollup.csv")

In [ ]:
cluster_data.head()

In [ ]:
cluster_data.describe()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

#Split data into train and test for clustering
X = cluster_data.drop(columns = "person_id")
y = cluster_data["person_id"]

#X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = .2, random_state = 1)

In [ ]:
#Performing kmeans and attempting different numbers of clusters

kmeans_per_k = [KMeans(n_clusters=k, random_state=42).fit(X)
                for k in range(1, 12)]

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_scores = [silhouette_score(X, model.labels_)
                     for model in kmeans_per_k[1:]]

In [ ]:
#Plotting the scores for each number of calculated Kmeans clusters

plt.figure(figsize=(8, 3))
plt.plot(range(1, 11), silhouette_scores, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Silhouette score", fontsize=14)
plt.axis([0, 12, 0.2, .8])
#save_fig("silhouette_score_vs_k_plot")
plt.show()

In [ ]:
inertias = [model.inertia_ for model in kmeans_per_k]

print(inertias)

In [ ]:
plt.figure(figsize=(8, 3.5))
plt.plot(range(1, 12), inertias, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Inertia", fontsize=14)
plt.axis([0, 12, 1000000, 40000000])
#save_fig("inertia_vs_k_plot")
plt.show()

Based on the inertia and the the silhouette scores of the different number of clusters, the best number of clusters is 2 or 3 clusters. 2 clusters have the largest silhouette score (not counting 1), and 3 is the elbow point of the models inertia and also has a higher silhouette score.

## Standardization of clusters

In [ ]:
from sklearn.preprocessing import StandardScaler
from pandas.plotting import scatter_matrix

scaler = StandardScaler()
scaled_model = scaler.fit_transform(X)


In [ ]:
X.head()

In [ ]:
log = pd.DataFrame(X)
log

In [ ]:

scatter_matrix(pd.DataFrame(scaled_model), figsize = (10,10))

In [ ]:
pd.DataFrame(scaled_model).head()

In [ ]:
#Performing kmeans and attempting different numbers of clusters

kmeans_per_k = [KMeans(n_clusters=k, random_state=42).fit(scaled_model)
                for k in range(1, 12)]

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_scores = [silhouette_score(scaled_model, model.labels_)
                     for model in kmeans_per_k[1:]]

In [ ]:
#Plotting the scores for each number of calculated Kmeans clusters

plt.figure(figsize=(8, 3))
plt.plot(range(1, 11), silhouette_scores, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Silhouette score", fontsize=14)
plt.axis([0, 12, 0.2, .6])
#save_fig("silhouette_score_vs_k_plot")
plt.show()

In [ ]:
inertias = [model.inertia_ for model in kmeans_per_k]

print(inertias)

In [ ]:
plt.figure(figsize=(8, 3.5))
plt.plot(range(1, 12), inertias, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Inertia", fontsize=14)
plt.axis([0, 12, 10000, 110000])
#save_fig("inertia_vs_k_plot")
plt.show()

### Clustering results

In [ ]:
kmeans2 = KMeans(n_clusters = 2)
kmeans3 = KMeans(n_clusters = 3)

kmeans2.fit_predict(scaled_model)
kmeans3.fit_predict(scaled_model)

labels2 = kmeans2.labels_
labels3 = kmeans3.labels_

cluster_data2 = cluster_data
cluster_data2['cluster_num'] = labels2
cluster_data2.head()


In [ ]:
#Group by the clusters 
# cluster_data2.groupby('cluster_num').mean()

In [ ]:
cluster_data3 = cluster_data
cluster_data3['cluster_num'] = labels3
cluster_data3.head()

In [ ]:
#Group by the clusters
# cluster_data3.groupby('cluster_num').mean()

In [ ]:
cluster_data3.cluster_num.value_counts()

## Linear Discriminant Analysis

Now we compute the linear discriminant analysis based off the clusters made previously

In [ ]:
#first we want to add in the discrimnant variables of the dataset. Age, income, gender, and when they became a member
profile = pd.read_csv("profile_transformed_1.csv")

#We remove the extra index and the scaled membership days and when they became a member
profile = profile.drop(columns = ["Unnamed: 0", "membership_length_scaled", "became_member_on", "became_member_on_converted"])
profile.head()

In [ ]:
#Merge the profile data to the cluster result data
lda_data = profile.merge(cluster_data3, how = "inner", left_on = "id", right_on = "person_id")

#remove repeat of id
lda_data = lda_data.drop(columns = "id")


In [ ]:
#since we don't have enough info on "other" in gender to create a conclusive analysis, we will remove it and convert gender into a binary variable
lda_data.gender.value_counts()

# converting to binary data for easier analysis
df_one = pd.get_dummies(lda_data["gender"])

 
# display result
df_two = pd.concat((df_one, lda_data), axis=1)
df_two = df_two.drop(["gender", 'O'], axis=1)
df_two = df_two.drop(["M"], axis=1)
#convert binary data so if gender = 1 is female and gender = 0 is male
lda_updated = df_two.rename(columns={"F": "gender"})

lda_updated.head()


In [ ]:
#Create X and y for lda 
#For LDA we want to focus on the discriminant variables and their relation to the cluster numbers
y = lda_updated.cluster_num
X = lda_updated[["gender", "age", "income", "membership_length_days"]]

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# create the lda model
lda = LinearDiscriminantAnalysis()

#Define the evaluation method using kfolds
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=1)

In [ ]:
lda.fit(X, y)

In [ ]:
# evaluate model
scores = cross_val_score(estimator = lda, X = X, y = y,
                         scoring='accuracy',
                         cv=cv, n_jobs=-1)
# summarize result
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

In [ ]:
# Evaluate result
means = pd.DataFrame(lda.means_, columns = ["gender", "age", "income", "membership_length_days"])

round(means, 3)

1. Age doesnt seem to influence behavior that much. 
2. Income does separate cluster 2 from the others.
3. Cluster 1 has longer membership
4. Cluster 0 is closer to male